# Social networks
We have a body of tweets from which we can understand who is communication with who and by extension which communities exist within our body of tweeters. We also have a set of tweeter screen names, labeled to express whether they hold an opinion 'AGAINST' or 'NOT AGAINST' the London Mayor on the causes of crime. In this notebook we will use this data 

## Context
Previously we analysed Twitter data to understand 'WHAT' was being discussed about serious violent crime on Twitter and used that to predict whether the opinion expressed within the tweet was 'AGAINST' or 'NOT AGAINST' the causes given by the Mayor. In this notebook we focus on the 'WHO', which is the communities that are discussing crime and the key influencers within these communities. Having identified these communities we can then see how which of these communities tweeters we previously labeled as against belong to and whether there is any relationship. This will help us answer our sec



This is the first step in helping us answer our research question
- Research Question 1 (RQ1): Can Twitter sentiment analysis determine the proportions of Twitter users that accept or reject the London Mayor’s evidence that ‘deprivation in the leading cause of youth violent crime in London’?
- Research Question 2 (RQ2): Can this approach additionally identify the social groups to which users rejecting the Mayor’s evidence belong, and whether this view is widely spread within those groups?





we were going to do membership over time but that was on the expectation we could get twitter data spanning 2 years. We don't believe this is beneficial given we only have data for 1 month.

Add as a conclusion:
Strongly connected networks are two way conversations but previously we saw 75% of tweets were retweets and so this suggests the networks are far more one way and so that's why we get more useful results using weakly connected networks.



I basically use two sources for this, Bovet and programminghistorian

What I want to find out is basically the following:
1. Are there obvious social clusters within our network
2. If so, how big are they and how strong are the relationships
3. Within these networks, who are the most important tweeters

We also want to know the following general information:
- Which users are most important, irrespective of network

## Changes I need to make
- Bring strong connections up front - say not really informative but keep an eye out for key players
- Then do overview of degrees and centrality - explain why
- Get components + communities
- Do page ranking - explain why and compare with degrees

In [ ]:
import datetime
import pandas as pd
import numpy as np

In [ ]:
all_tweets = pd.read_csv("./DataSources/TwitterData/cleaned_tweets.csv")
print(all_tweets.shape)
all_tweets.head()

In [ ]:
all_tweets = all_tweets.dropna(how='all') # only drops a row when every column is NA
print("shape before after dropping rows with all NaN")
print(all_tweets.shape)

# Now check for individual NaN values
nan_values = all_tweets[all_tweets.isna().any(axis=1)]
print(nan_values.count())

In [ ]:
# need to set the in_reply_to_user_screen_name and quote_tweet_screen_name fields to blanks
all_tweets.loc[all_tweets['in_reply_to_user_screen_name'].isna(), 'in_reply_to_user_screen_name'] = ''
all_tweets.loc[all_tweets['quote_tweet_screen_name'].isna(), 'quote_tweet_screen_name'] = ''

# Now check for individual NaN values
nan_values = all_tweets[all_tweets.isna().any(axis=1)]
print(nan_values.count())

nan_values.head()

## Building the network of interactions
### This code draws heavily on the work of Bovet
### https://github.com/alexbovet/network_lesson/blob/master/02_Analysis_of_Twitter_Social_Network.ipynb

We will use the python module NetworkX to construct and analyze the social network.

There are four types of interactions between two users in Twitter:
- Retweet
- Quote
- Reply
- Mention

In [ ]:
# let's define some functions to extract the interactions from tweets
def string_to_list(my_str):
    delimiter = ","
    my_str = my_str.replace("[", "")
    my_str = my_str.replace("]", "")
    my_str = my_str.replace("@", "")
    my_str = my_str.replace("'", "")
    my_str = my_str.replace(" ", "")
    my_list = my_str.split(delimiter)
    return my_list

def getAllInteractions(tweet):
    
    # Get the tweeter
    tweet_id = tweet.tweet_id
    tweeter_id = tweet.tweeter_id
    tweeter_name = tweet.tweeter_screen_name
    
    # a python set is a collection of unique items
    # we use a set to avoid duplicated ids
    interacting_users = set()
    
    # Add person they're replying to
    if tweet.in_reply_to_user_screen_name != '':
        interacting_users.add(tweet.in_reply_to_user_screen_name)
        
    # Add person they quoted
    if tweet.quote_tweet_screen_name != '':
        interacting_users.add(tweet.quote_tweet_screen_name)
    
    # Add person they retweeted
    if len(tweet.retweeted) > 2: # because empty strings will contain []
        retweeted_list = string_to_list(tweet.retweeted)
        for item in retweeted_list:
            interacting_users.add(item)
       
    # Add mentions
    if len(tweet.mentioned) > 2: # because empty strings will contain []
        mentioned_list = string_to_list(tweet.mentioned)
        for item in mentioned_list:
            interacting_users.add(item)
  
    # remove the tweeter if he is in the set
    interacting_users.discard(tweeter_name)
    
    # Return our tweeter and their influencers
    return tweeter_id, tweeter_name, tweet_id, list(interacting_users)

In [ ]:
import networkx as nx

# define an empty Directed Graph
# A directed graph is a graph where edges have a direction
# in our case the edges goes from user that sent the tweet to
# the user with whom they interacted (retweeted, mentioned or quoted)
G = nx.DiGraph()

for index, tweet in all_tweets.iterrows():
    
    if (tweet.tweeter_screen_name != 'SadiqKhan') & (tweet.tweeter_screen_name != 'MayorofLondon'):
        tweeter_id, tweeter_name, tweet_id, interactions = getAllInteractions(tweet)
    
        # add an edge to the Graph for each influencer
        for interact_name in interactions:
        
            # add edges between the two user ids
            # this will create new nodes if the nodes are not already in the network
            # we also add an attribute the to edge equal to the id of the tweet
            G.add_edge(tweeter_name, interact_name, tweet_id=tweet_id)
        
            # add name as a property to each node
            # with networkX each node is a dictionary
            G.nodes[tweeter_name]['name'] = tweeter_name
            G.nodes[interact_name]['name'] = interact_name

## Start with network characteristics
Based on tutorial from: https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python 
- John R. Ladd, Jessica Otis, Christopher N. Warren, and Scott Weingart, "Exploring and Analyzing Network Data with Python," The Programming Historian 6 (2017), https://doi.org/10.46430/phen0064.

In [ ]:
print(nx.info(G))

In [ ]:
# On a scale of 0 to 1, where 1 is a dense network
density = nx.density(G)
print("Network density:", density)

In [ ]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

In [ ]:
# the keys are the user_id
nodelist = list(G.nodes.keys())
print(nodelist[:10])

In [ ]:
# each node is itself a dictionary with node attributes as key,value pairs
print(type(G.nodes[nodelist[3]]))
print(G.nodes[nodelist[3]])

In [ ]:
from operator import itemgetter
from networkx.algorithms import community 

sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [ ]:
print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)

In [ ]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

In [ ]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

In [ ]:
#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

In [ ]:
H = nx.Graph(G) # need to convert to undirected graph to use greedy_modularity

communities = community.greedy_modularity_communities(H)

In [ ]:
modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'modularity')

In [ ]:
df_community_dict = pd.DataFrame(list(modularity_dict.items()),columns = ['screen_name','class_id']) 
class_list = df_community_dict.class_id.unique()

print("Number of different communities = {} ".format(len(class_list)))

df_community_dict_agg = df_community_dict.groupby('class_id').count()

comm_count = 10
print("\nTop {} communities, by number of members".format(comm_count))
df_community_dict_agg.head(comm_count)

In [ ]:
from matplotlib import pyplot as plt

def print_class_members(class_num, count):

    # First get a list of just the nodes in that class
    class_id = [n for n in G.nodes() if G.nodes[n]['modularity'] == class_num]

    # Then create a dictionary of the eigenvector centralities of those nodes
    class_eigenvector = {n:G.nodes[n]['eigenvector'] for n in class_id}

    # Then sort that dictionary and print the first 5 results
    class_sorted_by_eigenvector = sorted(class_eigenvector.items(), key=itemgetter(1), reverse=True)

    print("\n<--- Modularity Class {} Sorted by Eigenvector Centrality, top {} values --->".format(class_num, count))
    for node in class_sorted_by_eigenvector[:count]:
        print("Name:", node[0], "| Eigenvector Centrality:", node[1])
        
    my_subgraph = G.subgraph(class_id)
    
    plt.figure()
    nx.draw(my_subgraph)
    plt.show()


### Compare with classes allocated via labeled tweets

In [ ]:
labeled_users = pd.read_csv('./DataSources/TwitterData/labeled_users.csv')
print(labeled_users.shape)
print(labeled_users[labeled_users.tweeter_label=='AGAINST'].shape)
labeled_users.head()

In [ ]:
print(df_community_dict.shape)
df_community_dict.head()

In [ ]:
merged_df = pd.merge(labeled_users, df_community_dict, left_on='tweeter_screen_name', right_on='screen_name')
print(merged_df.shape)

In [ ]:
just_against = merged_df[merged_df.tweeter_label=='AGAINST'].copy()
against_count = just_against.shape[0]
print(just_against.shape)
just_against.head()

In [ ]:
just_against_sorted = just_against[['AGAINST','class_id']].groupby(['class_id'])['AGAINST'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) 

just_against_sorted['class_pct_of_total'] = just_against_sorted['count'] / against_count

just_against_sorted.reset_index().head(20)

### Comments
We have 5,011 in combined dataframe and approximately 65% of these fall into the classes: 0, 1, 9, 5, 2, 13, 4 and 11

Now we take a look at these classes to see their membership and whether they correlate.

n.b. group by logic above courtesy of https://stackoverflow.com/questions/40454030/count-and-sort-with-pandas

In [ ]:
class_list = [0, 1, 9, 5, 2, 13, 4, 11]
top_member_count = 10

for class_id in class_list:
    print("<----- class {} ----->".format(class_id))
    print_class_members(class_id, top_member_count)


In [ ]:
df_community_just_against = df_community_dict[df_community_dict.class_id.isin(class_list)].copy()

df_community_just_against_sorted = df_community_just_against.groupby('class_id').count().reset_index()
df_community_just_against_sorted.head(20)

In [ ]:
sorted_merged = pd.merge(df_community_just_against_sorted, just_against_sorted, left_on='class_id', right_on='class_id')
sorted_merged['pct_of_class'] = sorted_merged['count'] / sorted_merged['screen_name']

sorted_merged.rename(columns = {'screen_name':'community_count', 
                                'count':'label_count', 
                                'class_pct_of_total':'class_as_pct_all_against_labels',
                                'pct_of_class':'label_as_pct_whole_class'}, inplace = True)

sorted_merged.sort_values(by=['label_as_pct_whole_class'], ascending=False).head(20)

### Also need to see what proportion of NOT_AGAINST fall into these classes

## carry on with bovet

In [ ]:
# edges are contained in a EdgeView with a set-like interface
print(type(G.edges))
print(G.edges())

In [ ]:
# we can see all the edges going out of this node
# each edge is a dictionary inside this dictionary with a key 
# corresponding to the target user_id
e = G.out_edges(nodelist[4], data=True)
print(nodelist[4])
print(e)

In [ ]:
# we can iterate over the out-edges 
for s,t,data in e:
    print(s,t,data)

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
# dictionary with the degree of all nodes
all_degrees = [G.degree(n) for n in nodelist] # this is the degree for undirected edges
in_degrees = [G.in_degree(n) for n in nodelist]
out_degrees = [G.out_degree(n) for n in nodelist]

In [ ]:
# average degree
2*G.number_of_edges()/G.number_of_nodes()

In [ ]:
import numpy as np
np.array(all_degrees).mean()

In [ ]:
np.array(in_degrees).mean()

In [ ]:
np.array(out_degrees).mean()

In [ ]:
# we want to make a list with (user_id, username, degree) for all nodes
degree_node_list = []
for node in nodelist:
    degree_node_list.append((node, G.nodes[node]['name'], G.degree(node)))
    
print('Unordered user, degree list')    
print(degree_node_list[:10])

# sort the list according the degree in descinding order
degree_node_list = sorted(degree_node_list, key=lambda x:x[2], reverse=True)
print('Ordered user, degree list')    
print(degree_node_list[:10])

In [ ]:
# we need to import matplolib for making plots
# and numpy for numerical computations
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Network components
Connected components are a subset of nodes in which each node has a minimum of one link with another node in the same subset, and any node that is a member of 'this' subset is not linked to any nodes external to the subset. 

In addition, for <b> directed </b> graphs we can define two types of connected components:
- Weakly connected components, (WCC): maximal set of nodes where there exists a path in at least one direction between each pair of nodes.
- Strongly connected components, (SCC): maximal set of nodes where there exists a path in both directions between each pair of nodes.

And, within these categories we can additionally identify the weakly connected giant (WCGC), which is the largest of the weakly connected components and the strongly connected giant (SCGC), which is the largest of the strongly connected components. In effect these are the largest subgraphs within the network. Bovet et al illustrate this (incorrectly, for SCGC given arrows aren't two way) as follows:

<img src="WCGC-SCGC.png">

## Strongly connected components

In [ ]:
# this returns a list of set of nodes belonging to the 
# different (strongly) connected components
components_strong = list(nx.strongly_connected_components(G))

# sort the component according to their size
components_strong = list(sorted(components_strong, key=lambda x:len(x), reverse=True))

# make a list with the size of each component
comp_sizes_strong = []
for comp in components_strong:
    comp_sizes_strong.append(len(comp))
    
print("number of strong components: {}".format(len(comp_sizes_strong)))

print("sizes of the ten largest components: {}".format(comp_sizes_strong[:20]))

In [ ]:
def draw_strongly_connected(idx):

    largest_comp_strong = components_strong[idx]
    LCC_strong = G.subgraph(largest_comp_strong)

    print("number of components in LCC[{}] = {}".format(idx, LCC_strong.number_of_nodes()))

    # let's plot the degree distribution inside the LCC
    degrees = [LCC_strong.degree(n) for n in LCC_strong.nodes()]
    print("number of degrees for each node within LCC[{}] = {}\n".format(idx, degrees))
    
    # add weights to the edges based on how many times they are traversed
    print("Weight of edges between each node:\n")
    
    for u, v, d in LCC_strong.edges(data=True):
        d['weight'] = 1
    for u,v,d in LCC_strong.edges(data=True):
        print (u,v,d)
    
    ## <--- now look at centrality
    betweenValues = nx.betweenness_centrality(LCC_strong)
    
    # betweenValues is a dictionary, let's get the values and keys in separate lists
    values = list(betweenValues.values())
    keys = list(betweenValues.keys())
    
    # find the index of the node with highest betweeness centrality
    highestIndex = np.argmax(values)
    
    #Uses solution in the documentation: https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.components.connected.connected_component_subgraphs.html
    
    print('\n <---- centrality and clustering measures -------> \n')
    print("The node id ", keys[highestIndex], " has the centrality degree of ", values[highestIndex])

    overallAverage = []
    # now on to looking at clustering coefficients
    for i in range(0, countComps):
        clustCoeff = nx.clustering( connectedSubgraphs[i])
        coeffVals = list(clustCoeff.values())
        overallAverage.append(np.average(coeffVals))
    
    print ("Average clustering coefficent is: ", np.average(overallAverage))

    ## <--- end of this
    
    
    plt.figure()
    nx.draw_networkx(LCC_strong)
    plt.show()

In [ ]:
range_limit = 5
for n in range(range_limit):
    draw_strongly_connected(n)

## end of strongly connected components
- need to comment on interesting not necessarily for the size of the strong components but mostly because of the prominence of their members in other communities - look at that as part of the weakly connected components plus also the community

In [ ]:
# this returns a list of set of nodes belonging to the 
# different (weakly) connected components
components = list(nx.weakly_connected_components(G))

# sort the component according to their size
components = list(sorted(components, key=lambda x:len(x), reverse=True))

In [ ]:
# make a list with the size of each component
comp_sizes = []
for comp in components:
    comp_sizes.append(len(comp))

In [ ]:
# plot the histogram of component sizes
hist = plt.hist(comp_sizes, bins=100)

In [ ]:
# histogram with logarithmic y scale
hist = plt.hist(comp_sizes, bins=100, log=True)
tx = plt.xlabel('component size')
ty = plt.ylabel('number of components')

In [ ]:
# sizes of the ten largest components
comp_sizes[:10]

In [ ]:
# let's make a new graph which is the subgraph of G corresponding to 
# the largest connected component
# let's find the largest component
largest_comp = components[2]
LCC = G.subgraph(largest_comp)

In [ ]:
G.number_of_nodes()

In [ ]:
LCC.number_of_nodes()

In [ ]:
# let's plot the degree distribution inside the LCC
degrees = [LCC.degree(n) for n in LCC.nodes()]
degrees.sort(reverse=True)
degrees

In [ ]:
plt.figure(figsize=(10,10))
nx.draw_networkx(LCC)
plt.show()

In [ ]:

degree_array = np.array(degrees)
hist = plt.hist(degree_array, bins=100)

In [ ]:

# using logarithmic scales
hist = plt.hist(degree_array, bins=100, log=True)
plt.xscale('log')

In [ ]:
# logarithmic scale with logarithmic bins
N, bins, patches = plt.hist(degree_array, bins=np.logspace(0,np.log10(degree_array.max()+1), 20), log=True)
plt.xscale('log')
tx = plt.xlabel('k - degree')
ty= plt.ylabel('number of nodes')

In [ ]:
# Degree probability distribution (P(k))

# since we have logarithmic bins, we need to
# take into account the fact that the bins 
# have different lenghts when normalizing
bin_lengths = np.diff(bins) # lenght of each bin

summ = np.sum(N*bin_lengths)
normalized_degree_dist = N/summ

# check normalization:
print(np.sum(normalized_degree_dist*bin_lengths))

hist = plt.bar(bins[:-1], normalized_degree_dist, width=np.diff(bins))
plt.xscale('log')
plt.yscale('log')
tx = plt.xlabel('k (degree)')
ty = plt.ylabel('P(k)')

## Find most important Tweeters using Page Rank
PageRank is a generalisation of Google's websearch algorithm, which was originally a method for returning the most important web pages for a given search term. It defined importance as the page which - need to talk about teleporting and surfer stuff - reference this bloke: Gleich, D.F., 2015. PageRank beyond the web. Siam Review, 57(3), pp.321-363.

Also then look at from the perspective of centrality as this tells us which nodes(pages) have most edges in or out



In [ ]:
#teleportation probability
alpha = 0.15

#adjacency matrix
nodelist = list(G.nodes())
A = nx.to_numpy_array(G, nodelist=nodelist)

#diagonal matrix of out degrees
deg_out_vect = np.array([float(max(G.out_degree(n),1)) for n in nodelist])
D_out_inv = np.diag(1/deg_out_vect)

# teleportation transition matrix
N = A.shape[1]
S = np.ones((N,N))*1/N

# full transition matrix
M = (1-alpha)*D_out_inv @ A + alpha*S

# for dangling nodes (nodes without out-edges), we force the random teleportation
dangling_nodes = np.where(A.sum(1) == 0)[0]
M[dangling_nodes,:] = S[dangling_nodes,:]

#initial walker distribution and 1st iteration
p_last = np.ones(N)*1/N
p = np.matmul(p_last, M)

# iterate until sufficient convergence
eps = 1.0e-8
i = 1
while np.linalg.norm(p - p_last, 2) > eps:
        p_last = p
        p = np.matmul(p, M)
        i += 1

print(i)

In [ ]:
pg_ranking = np.array(np.argsort(p)[::-1])

pagerank_values = p[pg_ranking]
nodes_pagerank = [nodelist[r] for r in pg_ranking]
nodes_pagerank[:20]

In [ ]:
names_pagerank = [G.nodes[n]['name'] for n in nodes_pagerank]
names_pagerank[:20]

### Compare page rank with degree
Top 20 nodes by degree:
- ('KoolKat1025', 911)
- ('LeoKearse', 629)
- ('LeaveEUOfficial', 537)
- ('SadiqKhan', 510)
- ('MayorofLondon', 376)
- ('BrexitBassist', 317)
- ('PoliticsJOE_UK', 314)
- ('mariannaspring', 294)
- ('PoliticsForAlI', 240)
- ('LBC', 235)
- ('PrisonPlanet', 230)
- ('NKrankie', 219)
- ('talkRADIO', 210)
- ('TJ_Knight', 192)
- ('metpoliceuk', 171)
- ('ashindestad', 170)
- ('MickeyD44314901', 160)
- ('DJBURNS_was', 160)
- ('Independent', 160)
- ('standardnews', 152)

In [ ]:
hist = plt.bar(np.arange(p.shape[0]),np.sort(p)[::-1])
ty = plt.ylabel('PageRank value')
tx = plt.xlabel('PageRank ranking')

In [ ]:
# pagerank is a probability density
pagerank_values.sum()

In [ ]:
# draw the network of the top 5 nodes
plt.figure(figsize=(10,10))
nx.draw(G, nodelist=nodes_pagerank[:5], node_size=8000*pagerank_values[:5],width=0.5, arrows=False)
plt.show()

In [ ]:
# First let's add the pagerank value as a node attribute
for n, pr in zip(nodes_pagerank,pagerank_values):
    if n in LCC:
        LCC.nodes[n]['page_rank'] = pr

In [ ]:
nodes_pagerank[:5]